In [1]:
import pandas as pd
import numpy as np 

In [2]:
df = pd.read_csv('data.csv')

In [4]:
df.head()

,title,year,rating,director,cast,genre,plot,language,budget,gross,...,sound_department,sound_mix,special_effects,special_effects_companies,stunts,synopsis,transportation_department,visual_effects,votes,writers
0,Miss Jerry,1894,5.9,[<Person id:0085156[http] name:_Alexander Blac...,[<Person id:0063086[http] name:_Blanche Baylis...,"['Short', 'Romance']",['The adventures of a female reporter in the 1...,['None'],none,none,...,NaN,['Silent'],NaN,NaN,NaN,NaN,NaN,NaN,191.0,[<Person id:0085156[http] name:_Alexander Blac...
1,The Story of the Kelly Gang,1906,6.1,[<Person id:0846879[http] name:_Charles Tait_>],[<Person id:0846887[http] name:_Elizabeth Tait...,"['Action', 'Adventure', 'Biography', 'Crime', ...","['Story of Ned Kelly, an infamous 19th-century...",['None'],"$2,250 (estimated)",none,...,NaN,['Silent'],NaN,NaN,NaN,NaN,NaN,NaN,737.0,[<Person id:0846879[http] name:_Charles Tait_>]
2,Den sorte drøm,1911,5.9,[<Person id:0300487[http] name:_Urban Gad_>],[<Person id:0003425[http] name:_Asta Nielsen_>...,['Drama'],"[""Two men of high rank are both wooing the bea...",['None'],none,none,...,NaN,['Silent'],NaN,NaN,NaN,NaN,NaN,NaN,218.0,"[<Person id:0300487[http] name:_Urban Gad_>, <..."
3,Cleopatra,1963,7.0,[<Person id:0000581[http] name:_Joseph L. Mank...,[<Person id:0000072[http] name:_Elizabeth Tayl...,"['Biography', 'Drama', 'History', 'Romance']",['Queen Cleopatra VII of Egypt experiences bot...,['English'],"$31,115,000 (estimated)","$71,777,778",...,[<Person id:0179331[http] name:_James Corcoran...,"['70 mm 6-Track::(70 mm prints)', '4-Track Ste...",[<Person id:0096611[http] name:_Johnny Borgese...,NaN,"[<Person id:0042500[http] name:_Ray Austin_>, ...",['The movie opens in 48 B.C. shortly after the...,[<Person id:2170613[http] name:_Frank Khoury_>],"[<Person id:0008004[http] name:_L.B. Abbott_>,...",31526.0,[<Person id:0000581[http] name:_Joseph L. Mank...
4,Torment,1994,7.1,[<Person id:0001031[http] name:_Claude Chabrol_>],[<Person id:0000322[http] name:_Emmanuelle Béa...,"['Crime', 'Drama', 'Thriller']","[""Paul, an irritable and stressed-out hotel ma...",['French'],none,none,...,[<Person id:2694170[http] name:_Jean-Marie Bel...,"['Dolby SR', 'Stereo']",NaN,NaN,NaN,NaN,NaN,NaN,4936.0,[<Person id:0001031[http] name:_Claude Chabrol...


In [5]:
df.columns

Index(['title', 'year', 'rating', 'director', 'cast', 'genre', 'plot',
       'language', 'budget', 'gross', 'runtime', 'akas',
       'animation_department', 'art_department', 'art_directors',
       'aspect_ratio', 'assistant_directors', 'box_office',
       'camera_department', 'canonical_title', 'casting_department',
       'casting_directors', 'certificates', 'cinematographers', 'color_info',
       'composers', 'costume_departmen', 'costume_designers', 'countries',
       'country_codes', 'cover_url', 'distributors', 'editorial_department',
       'editors', 'full_size_cover_url', 'kind', 'language_codes',
       'location_management', 'long_imdb_canonical_title', 'long_imdb_title',
       'make_up_department', 'miscellaneous', 'music_department',
       'original_air_date', 'other_companies', 'plot_outline', 'producers',
       'production_companies', 'production_designers', 'production_managers',
       'script_department', 'set_decorators', 'smart_canonical_title',
       'sma

In [5]:
# df['genre'] = df['genre'].apply(apply_genre)
# mlist = []
# for i in range(len(df)):
#     mlist.extend(df.loc[i,'genre'])
# list(set(mlist))
# y = pd.get_dummies(df.genre, prefix = 'Genre')

In [6]:
def get_director_id(s):
    s1 = s
    s2 = 'Person id:'
    start = s1.index(s2) + len(s2)
    if s1[start:start+7].isnumeric():
        return s1[start:start+7]
    else:
        return np.nan

def get_director_name(s):
    s1 = s
    s2 = 'name:_'
    s3 = '_>]'
    start = s1.index(s2) + len(s2)
    end = s1.index(s3)
    try:
        return s1[start:end]
    except:
        return np.nan
    
def find(s, ch):
    l = []
    i = s.find(ch)
    while i!= -1:
        l.append(i)
        i = s.find(ch,i+1)
    return l

def get_cast_ids(s):
    start = find(s, '<Person id:')
    end = find(s, '[http]')
    id_list = []
    for i in range(len(end)):
        id_list.append(s[start[i]+11:end[i]])
    return id_list

def get_cast_names(s):
    start = find(s, 'name:_')
    end = find(s, '_>')
    name_list = []
    for i in range(len(end)):
        name_list.append(s[start[i]+6:end[i]])
    return name_list

def get_budget(s):
    start = s.find('$')
    end = s.find(' ')
    return int(s[start+1:end].replace(',',''))

def apply_genre(s):
    s = s.replace('\'','').replace('[','').replace(']','').split(sep = ', ')
    return s

def get_date(s):
    day = s[0:2]
    month = s[3:6]
    year = s[7:11]
    return day, month, year

def get_country(s):
    start = find(s, '(')
    end = find(s, ')')
    return s[start[0]+1:end[0]]

In [7]:
for i in df.index:
    s = df.loc[i,'director']
    try:
        df.loc[i,'director_name'] = get_director_name(s)
    except:
        df.loc[i,'director_name'] = np.nan
    try:
        df.loc[i,'director_id'] = get_director_id(s)
    except:
        df.loc[i,'director_name'] = np.nan
        
    s = df.loc[i,'cast']
    try:
        df.loc[i,'cast_names'] = [[get_cast_names(s)]]
    except:
        df.loc[i,'cast_names'] = np.nan
    try:
        df.loc[i,'cast_ids'] = [[get_cast_ids(s)]]
    except:
        df.loc[i,'cast_names'] = np.nan

for i in df.index:
    s = df.loc[i,'original_air_date']
    print(s)
    try:
        df.loc[i,'country'] = get_country(s)
        d,m,y = get_date(s)
        df.loc[i,'day'] = d 
        df.loc[i,'month'] = m
        df.loc[i,'year'] = y
    except:
        df.loc[i,'country'] = np.nan
        df.loc[i,'day'] = np.nan
        df.loc[i,'month'] = np.nan
        df.loc[i,'year'] = np.nan

In [9]:
df = df[['title','year','rating','director_name', 'genre','director_id', 'cast_names', 
         'cast_ids','votes','country','day','month']]

In [12]:
df.to_csv('selection_data.csv', index =False)

09 Oct 1894 (USA)
26 Dec 1906 (Australia)
19 Aug 1911 (Germany)
31 Jul 1963 (Canada)
16 Feb 1994 (France)
1913 (USA)
26 Nov 1919 (Denmark)
25 Dec 1951 (USA)
01 Sep 1912 (Romania)
20 Aug 1995 (Brazil)
nan
12 May 1913 (Finland)
27 Oct 1913 (Sweden)
08 Sep 1913 (Finland)
Jan 1914 (USA)
May 1914 (USA)
06 Nov 1913 (Mexico)
17 May 1914 (USA)
22 Aug 1913 (Germany)
24 Nov 1913 (USA)
nan
01 May 1916 (Finland)
24 Aug 1914 (USA)
21 Mar 1951 (France)
10 May 1957 (Turkey)
03 Sep 2021 (United Arab Emirates)
04 Aug 1913 (Denmark)
10 Jan 1914 (USA)
04 Sep 1916 (Sweden)
21 Feb 1921 (Finland)
16 Mar 1914 (Sweden)
14 Oct 1914 (USA)
20 Jan 1915 (USA)
07 Dec 1914 (USA)
08 Mar 1914 (USA)
28 Sep 1914 (USA)
24 Jul 1969 (Mexico)
15 Feb 1914 (USA)
20 Nov 1932 (USA)
21 Dec 1914 (USA)
nan
14 Jun 1987 (USA)
nan
09 Nov 1914 (USA)
08 Jun 1914 (USA)
15 Nov 1928 (USA)
12 Mar 2010 (Vietnam)
21 Mar 1915 (USA)
03 Oct 2003 (Spain)
nan
07 May 1965 (India)
12 Jan 1915 (USA)
30 Dec 1915 (USA)
04 Jan 1984 (France)
22 Sep 2005

06 Aug 1937 (USA)
03 Mar 2000 (UK)
16 Aug 1925 (USA)
1926 (Austria)
31 Jan 1926 (Finland)
1925 (Germany)
30 Apr 1926 (France)
15 Aug 1939 (Greece)
nan
28 Dec 1925 (USA)
27 Oct 1925 (Soviet Union)
02 May 1925 (USA)
25 Oct 1925 (Finland)
30 Sep 1926 (USA)
09 Aug 1959 (USA)
13 Sep 1926 (Canada)
1939 (UK)
19 Jul 1974 (West Germany)
23 Oct 1926 (USA)
1926 (Argentina)
08 Mar 1926 (USA)
nan
12 Sep 1926 (USA)
17 Oct 2008 (Austria)
17 May 1926 (USA)
02 May 1926 (USA)
26 Dec 1926 (Norway)
01 Mar 1926 (USA)
19 Sep 2013 (Russia)
06 Jun 1926 (USA)
13 Jun 1926 (USA)
14 Nov 1926 (USA)
02 Feb 2001 (Spain)
22 Aug 1926 (USA)
25 Apr 1979 (France)
25 Dec 1926 (USA)
05 Apr 1926 (USA)
11 Apr 1927 (Finland)
17 Oct 1926 (USA)
21 Jan 1985 (Poland)
Nov 1926 (Austria)
03 May 1927 (USA)
11 Jul 1926 (USA)
04 Oct 1926 (USA)
01 Apr 2016 (Spain)
24 Sep 1926 (Japan)
14 Feb 1927 (UK)
19 Mar 1926 (USA)
19 Oct 1925 (Finland)
29 Sep 2005 (Australia)
24 Jul 1926 (USA)
18 Feb 2016 (Italy)
18 Jan 2013 (Vietnam)
06 Feb 1927 (

23 Nov 1930 (USA)
20 Apr 1930 (USA)
05 Apr 1930 (USA)
24 May 1930 (USA)
28 Feb 1930 (USA)
14 Dec 1930 (USA)
31 Jan 2020 (Canada)
09 Aug 1930 (USA)
1930 (Turkey)
nan
17 Apr 1940 (USA)
15 May 1934 (France)
25 Jan 1931 (USA)
02 Feb 1930 (USA)
28 Feb 1930 (USA)
15 Oct 1930 (USA)
28 Nov 1930 (USA)
06 Sep 1930 (USA)
06 Apr 1930 (USA)
20 Sep 1930 (USA)
26 Mar 1930 (USA)
26 Aug 2005 (Denmark)
12 Apr 2017 (USA)
02 Aug 1930 (USA)
12 Jan 1935 (USA)
15 Jun 1930 (USA)
09 Feb 1930 (USA)
13 Apr 2012 (USA)
20 Sep 1930 (USA)
20 Mar 1930 (USA)
02 Sep 2011 (Vietnam)
06 Dec 1930 (Canada)
07 Nov 1930 (Ireland)
17 Jan 1931 (USA)
02 Feb 1930 (USA)
04 May 1930 (USA)
23 Aug 1930 (USA)
19 Oct 1930 (USA)
03 May 1930 (USA)
20 Mar 1942 (USA)
21 Oct 2006 (Portugal)
20 May 1944 (USA)
30 Dec 1930 (USA)
22 Apr 1930 (USA)
15 Jun 2001 (Italy)
25 Feb 1994 (South Korea)
05 Dec 1930 (France)
19 Jan 1930 (USA)
25 Aug 1930 (Soviet Union)
01 Aug 1930 (France)
Nov 1944 (USA)
14 Jan 1931 (USA)
11 Nov 1939 (Portugal)
15 Aug 1930

15 Feb 1932 (USA)
21 Feb 1932 (USA)
16 Dec 1932 (Japan)
04 Jun 1932 (USA)
30 Oct 1932 (USA)
nan
05 May 1932 (USA)
30 Dec 1932 (USA)
25 Nov 1932 (USA)
nan
07 Nov 1932 (UK)
20 Oct 1932 (USA)
15 Feb 1994 (USA)
25 Mar 1932 (USA)
22 Oct 1932 (USA)
29 Jan 1932 (USA)
06 Feb 1932 (USA)
07 Nov 1932 (USA)
09 Dec 1932 (USA)
15 Aug 1932 (USA)
23 Sep 1932 (USA)
14 Oct 1932 (USA)
07 Mar 1941 (USA)
25 May 1933 (USA)
27 Feb 1932 (USA)
22 Jan 1932 (USA)
18 Nov 1932 (USA)
23 Jul 1932 (USA)
12 Oct 1932 (USA)
24 Mar 1933 (USA)
22 Oct 1932 (USA)
21 Oct 1932 (USA)
25 Jun 1932 (USA)
21 May 1932 (USA)
13 Jun 1932 (UK)
27 Aug 1932 (USA)
06 May 1932 (USA)
08 Jul 1932 (USA)
27 Jan 1933 (France)
05 Dec 1932 (USA)
09 Apr 1932 (USA)
02 Dec 1932 (USA)
13 Oct 1932 (Japan)
13 May 1932 (Ireland)
01 Feb 1932 (USA)
31 Dec 2012 (Vietnam)
06 Aug 1943 (UK)
29 Dec 1987 (UK)
1933 (Austria)
27 Feb 1954 (USA)
22 Feb 1932 (USA)
13 May 1932 (USA)
16 Oct 1932 (USA)
16 Jul 1932 (USA)
24 Sep 1932 (USA)
30 Apr 1932 (USA)
23 Dec 1932 

01 Jun 1934 (USA)
20 Oct 1942 (USA)
1934 (Turkey)
10 Aug 2018 (Turkey)
20 Oct 1934 (USA)
24 Feb 1934 (USA)
20 Mar 1959 (USA)
15 Nov 1934 (USA)
1934 (Cuba)
30 Nov 1934 (USA)
15 Aug 1934 (USA)
1941 (Italy)
17 Mar 1934 (USA)
10 Mar 1934 (USA)
11 Apr 1984 (France)
26 Apr 1934 (Japan)
13 Oct 1934 (USA)
02 Mar 1934 (USA)
02 Nov 1934 (USA)
19 Oct 1983 (Italy)
10 Nov 1934 (USA)
15 Sep 1934 (USA)
13 Jul 1934 (USA)
12 Apr 1934 (Belgium)
10 Aug 1934 (USA)
15 Oct 1934 (USA)
13 Apr 1934 (USA)
22 Nov 1934 (USA)
06 Jul 1934 (USA)
02 Jan 1935 (USA)
22 Jun 1934 (USA)
15 May 1934 (France)
04 Jun 1934 (USA)
28 Dec 1934 (USA)
21 Mar 1980 (USA)
19 Jan 1934 (USA)
29 Mar 1934 (USA)
22 Mar 1934 (USA)
22 Mar 1934 (USA)
29 Sep 1934 (USA)
16 Feb 1934 (USA)
24 Oct 1936 (USA)
22 Jan 1934 (USA)
17 Nov 1934 (USA)
26 Nov 1919 (Denmark)
01 Jun 1956 (USA)
24 Dec 1934 (USA)
15 May 1934 (USA)
12 Dec 1963 (UK)
10 Feb 1934 (USA)
04 May 1934 (USA)
nan
14 Dec 1934 (France)
26 Oct 1934 (USA)
1934 (Turkey)
20 Jan 1934 (USA)
05

...

17 Jan 2019 (Kuwait)
16 Nov 2018 (Vietnam)
12 Apr 2019 (Australia)
09 Jan 2020 (United Arab Emirates)
05 Apr 2018 (South Korea)
11 Sep 2020 (UK)
24 Jul 2020 (Australia)
27 Sep 2019 (United Arab Emirates)
24 Apr 2019 (France)
nan
13 Sep 2019 (USA)
30 Jun 2018 (USA)
09 Nov 2018 (Poland)
08 Feb 2019 (Poland)
05 Sep 2003 (India)
10 Aug 2018 (Vietnam)
16 Mar 2007 (UK)
09 Apr 1999 (USA)
nan
04 Jan 2019 (Vietnam)
28 Mar 2018 (South Korea)
30 Jan 2019 (South Korea)
28 Dec 2018 (UK)
27 Nov 2019 (South Korea)
2018 (UK)
24 Aug 2018 (Norway)
26 Jul 2018 (Portugal)
24 Dec 1999 (South Korea)
06 Dec 2018 (Brazil)
09 Oct 2009 (Canada)
28 Jun 2019 (Australia)
22 Aug 1986 (Canada)
08 Mar 2019 (Australia)
22 Jun 2018 (Romania)
24 May 2018 (USA)
26 Mar 2020 (Italy)
19 Sep 2020 (USA)
16 Sep 2019 (Ireland)
22 Feb 2021 (South Korea)
20 Sep 2019 (Turkey)
07 Sep 2018 (India)
01 May 2019 (France)
09 Mar 2018 (UK)
12 Apr 2018 (Singapore)
22 Feb 2019 (USA)
nan
01 Sep 2020 (Turkey)
13 Sep 2019 (Australia)
20 Feb 2

2018 (India)
12 Jul 2019 (USA)
25 May 2018 (India)
30 May 2018 (Belgium)
23 Mar 2021 (Canada)
06 Mar 2019 (Belgium)
29 May 2020 (Taiwan)
25 May 2018 (Vietnam)
27 Jun 2019 (Ukraine)
nan
08 Jun 2018 (Turkey)
13 Feb 2020 (Serbia)
04 Oct 2013 (Vietnam)
18 May 2018 (China)
20 Dec 2019 (Vietnam)
02 Aug 2018 (Argentina)
01 Feb 2019 (India)
28 Feb 2020 (Vietnam)
08 Nov 2019 (Spain)
22 Nov 2019 (Spain)
07 Jun 2018 (Italy)
nan
20 Sep 2018 (India)
22 Apr 1969 (West Germany)
2018 (USA)
10 Aug 2018 (Australia)
13 Jun 2018 (France)
20 Sep 2019 (Australia)
25 Jun 2019 (Dominican Republic)
07 Feb 2019 (United Arab Emirates)
11 May 2007 (India)
15 Jul 2005 (Spain)
nan
10 Jan 2020 (Vietnam)
28 Jul 2018 (USA)
25 May 2018 (Turkey)
03 May 2019 (UK)
27 Dec 2018 (Italy)
21 May 2021 (UK)
06 Oct 1994 (Spain)
nan
23 Mar 1990 (USA)
2021 (UK)
24 Apr 2020 (USA)
11 Oct 2019 (Sweden)
12 May 2020 (Canada)
05 Apr 2019 (USA)
12 Oct 2018 (India)
07 Oct 2010 (Chile)
11 Oct 2018 (Australia)
20 Jul 2017 (Kuwait)
27 Jul 201

28 Dec 2018 (Taiwan)
26 Aug 2005 (Canada)
nan
20 Nov 2019 (France)
12 Jun 2019 (France)
07 Sep 2018 (Vietnam)
nan
1997 (Romania)
16 Oct 2020 (Canada)
04 Apr 2019 (USA)
29 Aug 2018 (USA)
26 Jul 2018 (Sweden)
28 Sep 2018 (Poland)
02 Aug 2020 (USA)
18 Apr 2019 (Austria)
13 Feb 2019 (Italy)
05 May 2018 (Japan)
22 Apr 2019 (Norway)
17 May 2019 (Australia)
22 Mar 2019 (India)
23 Apr 2020 (Hungary)
30 Aug 2018 (Russia)
nan
05 Dec 2019 (Kuwait)
07 Jun 2019 (Spain)
nan
26 Feb 2021 (China)
17 Apr 2019 (Belgium)
19 Jun 2020 (India)
12 Jun 2020 (France)
22 Nov 2019 (India)
14 May 2019 (Australia)
12 Oct 2018 (Turkey)
23 Nov 2018 (Turkey)
05 Feb 2019 (Brazil)
27 Jul 2018 (Australia)
nan
29 Dec 2017 (Turkey)
01 Nov 2017 (Romania)
08 Aug 2019 (Australia)
25 Jan 2019 (UK)
30 Mar 2019 (USA)
05 Oct 2019 (USA)
23 Aug 2019 (Norway)
29 Nov 2018 (Brazil)
09 Aug 2018 (Australia)
09 Aug 2018 (Brunei)
30 Mar 2019 (USA)
11 Apr 2020 (Brazil)
13 Nov 2018 (USA)
24 Jan 2019 (Czechia)
10 Oct 2018 (Belgium)
nan
09 Au

01 Nov 2019 (Vietnam)
29 Aug 2019 (Czechia)
03 Feb 2019 (USA)
23 Dec 2020 (Spain)
22 May 2020 (Vietnam)
21 Feb 2019 (Canada)
nan
12 Sep 2019 (Italy)
06 Sep 2019 (Australia)
03 May 2019 (Australia)
18 Oct 2019 (United Arab Emirates)
03 May 2019 (UK)
25 Sep 2018 (China)
05 Dec 2019 (Russia)
01 May 2014 (UK)
12 Jun 2019 (France)
25 Jan 2019 (Turkey)
nan
23 Oct 2020 (Sweden)
08 Mar 2019 (Finland)
21 Jan 2021 (Spain)
04 Nov 2014 (USA)
06 Aug 2020 (Canada)
26 Oct 2018 (Turkey)
04 Oct 2018 (Argentina)
18 Sep 2020 (Canada)
27 Sep 2018 (Malaysia)
Oct 2018 (Romania)
06 Sep 2019 (Spain)
04 Nov 2005 (Canada)
06 Feb 2019 (France)
09 Sep 2020 (France)
16 Aug 2019 (Poland)
03 Oct 2014 (Vietnam)
nan
27 Nov 2019 (Indonesia)
10 Jul 2020 (Australia)
01 Nov 2019 (Brazil)
29 May 1947 (Mexico)
24 Jul 2019 (USA)
26 Jun 2020 (Canada)
28 Mar 2019 (Hong Kong)
30 Jan 2004 (USA)
06 Feb 2019 (Canada)
nan
08 Nov 2018 (New Zealand)
26 Jul 2019 (Mexico)
27 Feb 2019 (UK)
19 Sep 2019 (Argentina)
18 Oct 2019 (India)
26 

07 Nov 2019 (Ukraine)
25 Feb 2020 (USA)
15 Feb 2019 (Turkey)
03 Jan 2019 (Indonesia)
24 Jul 2020 (UK)
08 Mar 2019 (Turkey)
07 Nov 2019 (Portugal)
28 Nov 1997 (India)
05 Jun 2019 (India)
12 Apr 2019 (Turkey)
02 Jul 2020 (Australia)
24 Jan 2019 (Russia)
20 Nov 2019 (France)
13 Feb 2020 (Israel)
13 Dec 2018 (Canada)
02 May 2019 (Argentina)
16 Nov 2018 (India)
11 Mar 2020 (Philippines)
21 Jun 2019 (India)
02 Apr 2020 (USA)
12 Sep 1996 (Germany)
12 Nov 1982 (Canada)
30 Jun 2020 (South Korea)
01 Nov 2019 (India)
06 Aug 2020 (Netherlands)
18 Jan 2019 (India)
28 Mar 2019 (India)
01 Oct 2019 (USA)
20 Dec 1950 (Denmark)
21 Dec 2018 (Turkey)
21 Dec 2018 (Sweden)
20 Dec 2018 (Russia)
02 Oct 2019 (France)
28 Feb 2019 (Australia)
05 Dec 2019 (Portugal)
10 Jul 2020 (Brazil)
nan
10 Jul 2019 (Canada)
29 Dec 2018 (China)
03 Aug 2007 (USA)
22 Nov 2019 (Australia)
15 Mar 2019 (India)
03 Aug 2018 (China)
15 Feb 2019 (Vietnam)
22 Nov 2019 (Turkey)
23 Aug 2019 (Vietnam)
06 Mar 2020 (Poland)
21 Feb 2020 (Japa

In [34]:
df['country']

0              USA
1        Australia
2          Germany
3           Canada
4           France
           ...    
85256    Australia
85257          USA
85258      Lebanon
85259          USA
85260       Norway
Name: country, Length: 85261, dtype: object